In [87]:
import altair as alt
cars = 'https://cdn.jsdelivr.net/npm/vega-datasets@1/data/cars.json'
movies = 'https://cdn.jsdelivr.net/npm/vega-datasets@1/data/movies.json'
sp500 = 'https://cdn.jsdelivr.net/npm/vega-datasets@1/data/sp500.csv'
stocks = 'https://cdn.jsdelivr.net/npm/vega-datasets@1/data/stocks.csv'

## <font color="#747a7f">6.5</font> Navegação: Visão geral + Detalhes


Ao mover e dar zoom, ajustamos diretamente a "área de visualização" de um gráfico. A estratégia de navegação relacionada de *visão geral + detalhes*, ao invés disso, usa uma exibição de visão geral para mostrar *tudo* dos dados, enquanto suporta seleções que movimentam e ampliam uma exibição de foco separada.

Abaixo, temos dois gráficos de área que mostram uma década de flutuações de preços para o índice de ações S&amp;P 500. Inicialmente, ambos os gráficos mostram o mesmo intervalo de dados. *Clique e arraste no gráfico de visão geral inferior para atualizar a exibição de foco e examinar intervalos de tempo específicos.*

In [88]:
brush = alt.selection_interval(encodings=['x']);

base = alt.Chart().mark_area().encode(
    alt.X('date:T', title=None),
    alt.Y('price:Q')
).properties(
    width=700
)

alt.vconcat(
    base.encode(alt.X('date:T', title=None, scale=alt.Scale(domain=brush))),
    base.add_params(brush).properties(height=60),
    data=sp500
)

alt.VConcatChart(...)

<!--Unlike our earlier panning &amp; zooming case, here we don't want to bind a selection directly to the scales of a single interactive chart. Instead, we want to bind the selection to a scale domain in _another_ chart. To do so, we update the `x` encoding channel for our focus chart, setting the scale `domain` property to reference our `brush` selection. If no interval is defined (the selection is empty), Altair ignores the brush and uses the underlying data to determine the domain. When a brush interval is created, Altair instead uses that as the scale `domain` for the focus chart.-->

Ao contrário do nosso conjunto panorâmica e zoom, aqui não queremos vincular uma seleção diretamente às escalas de um único gráfico interativo. Para isso, queremos vincular a seleção a um domínio de escala em _outro_ gráfico. Para fazer isso, atualizamos o canal de codificação `x` para nosso gráfico principal, definindo a propriedade de escala `domain` (domínio) para referenciar nossa seleção de `brush` (Pincel). Se nenhum intervalo for definido (seleção vazia), o Altair ignora o pincel e usa os dados subjacentes para determinar o domínio. Quando um intervalo de pincel é criado, o Altair o usa como o `domain` da escala para o gráfico principal.

## <font color="#747a7f">6.6</font> Detalhes sob demanda

Após nós identificarmos pontos de interesse em uma visualização, por vezes nós queremos saber mais sobre eles. *Detalhes sob demanda* refere-se à consultar interativamente por mais informações sobre os valores selecionados. *Tooltips* são um método útil de oferecer detalhes sob demanda. Entretanto, as tooltips normalmente mostram informações apenas para um ponto por vez. Como nós poderíamos mostrar mais?

O scatter plot das classificações de filmes inclui uma série de potenciais valores discrepantes interessantes onde as classificações do *Rotten Tomatoes* e do *IMDB* divergem. Vamos criar um gráfico que nos permita selecionar pontos interativamente e mostrar seus rótulos. Para acionar a consulta de filtro na interação de passar o mouse ou clicar, usaremos o [operador de composição Altair](https://altair-viz.github.io/user_guide/interactions.html#composing-multiple-selections) `|` ("ou").

*Passe o cursor sobre os pontos no scatter plot abaixo para ver um destaque e um rótulo. Clique com a tecla Shift pressionada para tornar as anotações persistentes e visualizar vários rótulos de uma só vez. Quais filmes são amados pelos críticos do Rotten Tomatoes, mas não pelo público em geral no IMDB (ou vice-versa)? Veja se consegue encontrar possíveis erros, onde dois filmes diferentes com o mesmo nome foram acidentalmente combinados!*

In [89]:
hover = alt.selection_point(
    on='mouseover',  # selecionar no mouseover
    nearest=True,    # seleciona o ponto mais próximo do cursor
    empty='none',    # seleção vazia não deve corresponder a nada
    toggle=False     # remove o shift + mouseover
)

click = alt.selection_point(
    empty='none' # seleção vazia não corresponde a nenhum ponto
)

# codificações do scatter plot compartilhadas por todas as marcas
plot = alt.Chart().mark_circle().encode(
    x='Rotten_Tomatoes_Rating:Q',
    y='IMDB_Rating:Q'
)

# base compartilhada por novas camadas
base = plot.transform_filter(
    hover | click # filtrar os pontos em ambas as direções
)

# camada com pontos do scatter plot, anotação circular, e títulos dos rótulos
alt.layer(
    plot.add_params(hover).add_params(click),
    base.mark_point(size=100, stroke='firebrick', strokeWidth=1),
    base.mark_text(dx=4, dy=-8, align='right', stroke='white', strokeWidth=2).encode(text='Title:N'),
    base.mark_text(dx=4, dy=-8, align='right').encode(text='Title:N'),
    data=movies
).properties(
    width=600,
    height=450
)

alt.LayerChart(...)

O exemplo acima adiciona três novas camadas ao scatter plot: uma anotação circular, texto branco para fornecer um fundo legível, e texto preto mostrando o título de um filme. Além disso, este exemplo usa duas seleções em conjunto:

1. Uma seleção única (`hover`) que inclui `nearest=True` para selecionar automaticamente o ponto mais próximo conforme o cursor se move.
2. Uma seleção múltipla (`click`) para criar seleções persistentes via shift-click.

Ambas as seleções incluem o conjunto `empty='none'` para indicar que nenhum ponto deve ser incluído se uma seleção estiver vazia. Estas seleções são então combinadas em uma único predicado &mdash; o *ou* lógico de `hover` e `click` &mdash; para incluir pontos que residem em *qualquer* seleção. Nós usamos esta condição para filtrar as novas camadas para mostrar anotações e rótulos apenas para pontos selecionados.

Usando seleções e camadas, nós podemos realizar vários designs diferentes para obter detalhes sob demanda! Por exemplo, aqui está uma série temporal em escala logarítmica de preços de ações de tecnologia, anotada com uma diretriz e categorias para a data mais próxima do cursor:

In [90]:
# seleciona um ponto para o qual oferece detalhes sob demanda
label = alt.selection_point(
    encodings=['x'], # limita a seleção ao valor do eixo x
    on='mouseover',  # seleciona em eventos mouseover
    nearest=True,    # seleciona o ponto mais próximo do cursor
    empty='none'     # seleção vazia não inclui nenhum ponto
)

# define nosso gráfico de linha dos preços das ações
base = alt.Chart().mark_line().encode(
    alt.X('date:T'),
    alt.Y('price:Q', scale=alt.Scale(type='log')),
    alt.Color('symbol:N')
)

alt.layer(
    base, # gráfico de linha base

    # adiciona uma marca para servir como linha guia
    alt.Chart().mark_rule(color='#aaa').encode(
        x='date:T'
    ).transform_filter(label),

    # adiciona marcas circulares para pontos de tempo selecionados, oculta pontos não selecionados
    base.mark_circle().encode(
        opacity=alt.condition(label, alt.value(1), alt.value(0))
    ).add_params(label),

    # adiciona texto branco para oferecer um fundo legível para rótulos
    base.mark_text(align='left', dx=5, dy=-5, stroke='white', strokeWidth=2).encode(
        text='price:Q'
    ).transform_filter(label),

    # adiciona rótulos de texto para o preços das ações
    base.mark_text(align='left', dx=5, dy=-5).encode(
        text='price:Q'
    ).transform_filter(label),

    data=stocks
).properties(
    width=700,
    height=400
)

alt.LayerChart(...)

*Colocando em ação o que nós aprendemos até agora: você pode modificar o scatter plot do filme acima (aquele com a consulta dinâmica ao longo dos anos) para incluir uma marca `rule` que mostra a classificação média do IMDB (ou Rotten Tomatoes) para os dados contidos na seleção `interval` do ano?*

## <font color="#747a7f">6.7</font> Brushing & Linking, Revisão

Anteriormente neste notebook nós vimos um exemplo de *brushing & linking*: usando um histograma de consulta dinâmica para destacar pontos em um scatter plot de classificação de filme. Aqui, nós visitaremos alguns exemplos adicionais envolvendo seleções vinculadas.

Voltando ao dataset `cars`, nós podemos usar o operador `repeat` para construir uma [matriz de scatter plot (SPLOM)](https://en.wikipedia.org/wiki/Scatter_plot#Scatterplot_matrices) que mostra associações entre quilometragem, aceleração e cavalos de potência. Nós podemos definir uma seleção  `interval` e incluí-la *dentro* de nossa especificação de scatter plot repetido para permitir seleções vinculadas entre todos os gráficos.

*Clique e arraste em qualquer uma das visualizações abaixo para realizar o brushing & linking!*

In [91]:
brush = alt.selection_interval(
    resolve='global' # resolve todas as seleções para uma única instância global
)

alt.Chart(cars).mark_circle().add_params(
    brush
).encode(
    alt.X(alt.repeat('column'), type='quantitative'),
    alt.Y(alt.repeat('row'), type='quantitative'),
    color=alt.condition(brush, 'Cylinders:O', alt.value('grey')),
    opacity=alt.condition(brush, alt.value(0.8), alt.value(0.1))
).properties(
    width=140,
    height=140
).repeat(
    column=['Acceleration', 'Horsepower', 'Miles_per_Gallon'],
    row=['Miles_per_Gallon', 'Horsepower', 'Acceleration']
)

alt.RepeatChart(...)

Observe acima o uso de `resolve='global'` na seleção `interval`. A configuração padrão `'global'` indica que em todos os gráficos apenas um brush pode estar ativo por vez. Entretanto, em alguns casos, nós poderíamos querer definir brushes em vários gráficos e combinar os resultados. Se nós usarmos `resolve='union'`, a seleção será a *união* de todos os brushes: se um ponto estiver dentro de qualquer brush ele será selecionado. Alternativamente, se usarmos `resolve='intersect'`, a seleção consistirá na *interseção* de todos os brushes: apenas os pontos que estão dentro de todos os brushes serão selecionados.

*Tente definir o parâmetro `resolve` para `'union'` e `'intersect'` e veja como isso muda a lógica da seleção resultante.*